1. Data Extraction and filtering

In [1]:
# libraries
import pandas as pd
import re
from collections import defaultdict

In [2]:
#function for filtering the whatsapp exported file 
def preprocess(file_path:str)->pd.DataFrame:
    encryption_message="Messages and calls are end-to-end encrypted. Only people in this chat can read, listen to, or share them. Learn more.\n"
    grppermission=r"\s*(.*) changed this group's settings to allow \s*(.*)"
    media_pattern="<Media omitted>"
    email_pattern=r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}"
    url_pattern=r"https?://[^\s]+"
    edited_message="<This message was edited>"
    deleted_message="You deleted this message"
    grp_deleted="This message was deleted"
    null_message="null"
    created_group="created group"
    added_you="added you"
    tagging_pattern=r'@[\w]+'
    noti=r"Your security code with \s*(.*)"
    no_change=r"\s*(.*) changed their phone number to a new number. Tap to message or add the new number."
    vcfmsg=r".*?\.vcf \s*\(file attached\)"
    
    with open(file_path,'r',encoding='utf-8') as f:
        lines=f.readlines()
    filtered_lines=[]
    for line in lines:
        if(encryption_message not in line and
           deleted_message not in line and
           grp_deleted not in line and
           null_message != line.split(" ")[-1] and
           media_pattern not in line and
           created_group not in line and
           added_you not in line and
           not re.search(email_pattern,line) and
           not re.search(url_pattern,line) and 
           not re.search(noti,line) and 
           not re.search(no_change,line) and not re.search(vcfmsg,line) and not re.search(grppermission,line)):
            line=line.replace(edited_message,"").strip()
            line=re.sub(tagging_pattern,"",line).strip()
            filtered_lines.append(line)
    content='\n'.join(filtered_lines)
    content=content.replace('\u202f'," ")#clean invisible lines
    # print(content)
    
    #regex for message 
    
    pattern= r'(\d{2}/\d{2}/\d{4}), (\d{1,2}:\d{2}\s?[APMapm]{2}) - ([^:]+):\s*(.*)'
    pattern24= r'(\d{2}/\d{2}/\d{4}), (\d{2}:\d{2}) - ([^:]+):\s*(.*)'
    messages=re.findall(pattern,content)
    messages24=re.findall(pattern24,content)
    # print(messages)
    # print(messages24)
    combined=[(f"{d}, {t}",s,m) for d,t,s,m in messages]
    combined+=[(f"{d}, {t}",s,m) for d,t,s,m in messages24]
    df=pd.DataFrame(combined,columns=['timestamp','sender','message'])
    # df.insert(0,'id',range(0,len(df)))
    # df.set_index('id', inplace=True)
    #handling timestamp
    
    timestamps=[]
    for timestamp in df['timestamp']:
        try:
            timestamp=pd.to_datetime(
                timestamp,errors='coerce'
            )
        except Exception as e:
            print("error",e)
            timestamp=pd.NaT
        timestamps.append(timestamp)
    df['timestamp']=timestamps
    return df

In [3]:
file=preprocess("./dataset/WhatsApp Chat with Didi Jio.txt")
file2=preprocess("./dataset/WhatsApp Chat with Kartikey Dubey.txt")
file3=preprocess("./dataset/WhatsApp Chat with Akansha Shishodia.txt")
file4=preprocess("./dataset/WhatsApp Chat with I.T. 3rd year unofficial.txt")
file5=preprocess("./dataset/WhatsApp Chat with Hack it.txt")
file6=preprocess("./dataset/WhatsApp Chat with IT -III YEAR (LATERAL ENTRY) 2023-2026.txt")
file7=preprocess("./dataset/WhatsApp Chat with Mami.txt")
file8=preprocess("./dataset/WhatsApp Chat with IT Information.txt")
file9=preprocess("./dataset/WhatsApp Chat with Placement Information batch 2023.txt")
file10=preprocess("./dataset/whatsapp-chat-data.txt")
dataset=pd.concat([file,file2,file3,file4,file5,file6,file7,file8,file9,file10])
dataset.to_csv("./dataset/chat.csv",index=False)


C:\Users\Admin\AppData\Local\Temp\ipykernel_2904\3372683218.py:59: UserWarning: Parsing dates in %d/%m/%Y, %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  timestamp=pd.to_datetime(
C:\Users\Admin\AppData\Local\Temp\ipykernel_2904\3372683218.py:59: UserWarning: Parsing dates in %d/%m/%Y, %H:%M am format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  timestamp=pd.to_datetime(
C:\Users\Admin\AppData\Local\Temp\ipykernel_2904\3372683218.py:59: UserWarning: Parsing dates in %d/%m/%Y, %H:%M pm format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  timestamp=pd.to_datetime(


In [4]:
dataset.shape

(38980, 3)

output

In [5]:
text_seq=(" ").join(dataset['message'])
with open('./output/text_seq.txt','w',encoding='utf-8') as f:
    f.write(text_seq)

manual bpe

In [ ]:
# def preprocess_text(text_seq):
#     text_seq.lower()
#     texta_seq=re.sub(r'[^\w\s]','',text_seq)
#     texts=text_seq.split()
#     return texts
# #intial vocab
# def get_vocab(corpus):
#     vocab = defaultdict(int) # defautdict is a data structure which help in avoiding key error if unique word enter then the intialize its freq to zero so we can easily increase it
#     for word in corpus:
#         chars = ' '.join(list(word)) + ' </w>'  # Add </w> to mark end of word
#         vocab[chars] += 1
#     return vocab
# # vocab=get_vocab(texts)
# # print(len(vocab))
# #count freq of adjecent symbol pairs
# def get_stats(vocab):
#     pairs=defaultdict(int)
#     for word,freq in vocab.items():
#         symbols=word.split()
#         for i in range(len(symbols)-1):
#             pair=(symbols[i],symbols[i+1])
#             pairs[pair]+=freq
#     return max(pairs,key=pairs.get), max(pairs.values())
# # pair=get_stats(vocab)
# # print(pair)
# #merge
# def merge_vocab(pair, vocab):
#     pattern = re.escape(' '.join(pair))
#     replacement = ''.join(pair)
#     new_vocab = {}
#     for word in vocab:
#         new_word = re.sub(pattern, replacement, word)
#         new_vocab[new_word] = vocab[word]
#     return new_vocab
# # result=merge_vocab(pair[0],vocab)
# # print(result)
# def bpe(corpus, num_merges=10):
#     vocab = get_vocab(corpus)
#     print("Initial Vocabulary:")
#     for word in vocab:
#         print(word)
#     print("\nStarting BPE Merges:\n")
#     for i in range(num_merges):
#         pair, freq = get_stats(vocab)
#         if not pair:
#             break
#         print(f"Merge #{i+1}: {pair} -> {''.join(pair)} (Frequency: {freq})")
#         vocab = merge_vocab(pair, vocab)
#     return vocab
# corpus=preprocess_text(text_seq)
# result=bpe(corpus,num_merges=11)
# result

Initial Vocabulary:
A a r a m </w>
s e </w>
p a h u n c h </w>
g y e e </w>
n n </w>
T a b i y a t </w>
k a i s i </w>
h h </w>
H a </w>
T h i k </w>
h </w>
A x x a a </w>
A a t e </w>
w a q t </w>
k h a s i </w>
k i </w>
s y r u p </w>
l e t e </w>
a a n a a </w>
J o </w>
r k h a a </w>
h u a a </w>
t m h a r e </w>
p e </w>
T o </w>
j o i n </w>
t h e </w>
m e e t i n g </w>
o n </w>
G o o g l e </w>
M e e t </w>
c l i c k </w>
t h i s </w>
l i n k </w>
A t </w>
2 </w>
4 5 </w>
p m </w>
t o d a y </w>
y o u </w>
p o s t </w>
m e s s a g e </w>
T B </w>
a d o p t i o n </w>
i s </w>
g o o d </w>
i d e a </w>
p a t i e n t </w>
a r e </w>
p r o n e </w>
t o </w>
C O V I D </w>
1 9 </w>
W e </w>
s t u d e n t s </w>
o f </w>
P h a r m a c y </w>
d e d i c a t e d </w>
e r a d i c a t e </w>
L i v e </w>
T i s s u e </w>
c h a h i y e </w>
t h a a </w>
म न न य </w>
क ल प त </w>
ज </w>
क </w>
न र द श न </w>
म </w>
ट ब </w>
स </w>
ग र स त </w>
र ग य </w>
द ख भ ल </w>
व </w>
उ न क </w>
ड ट 

{'A a r a m </w>': 12,
 's e</w>': 1168,
 'p a h u n c h</w>': 7,
 'g y e e</w>': 2,
 'n n </w>': 9,
 'T a b i y a t</w>': 6,
 'k a i s i</w>': 16,
 'h h</w>': 78,
 'H a</w>': 458,
 'T h i k </w>': 80,
 'h</w>': 3821,
 'A x x a a</w>': 7,
 'A a t e</w>': 2,
 'w a q t</w>': 2,
 'k ha s i</w>': 3,
 'k i</w>': 1155,
 's y r u p </w>': 3,
 'l e t e</w>': 57,
 'a a n a a</w>': 1,
 'J o</w>': 155,
 'r k ha a</w>': 1,
 'h u a a</w>': 7,
 't m ha r e</w>': 6,
 'p e</w>': 356,
 'T o</w>': 103,
 'j o in </w>': 93,
 't h e</w>': 1825,
 'm e e t in g </w>': 54,
 'o n </w>': 428,
 'G o o g l e</w>': 89,
 'M e e t</w>': 6,
 'c l i c k </w>': 33,
 't h i s</w>': 792,
 'l in k </w>': 213,
 'A t</w>': 19,
 '2 </w>': 248,
 '4 5 </w>': 20,
 'p m </w>': 492,
 't o d a y </w>': 84,
 'y o u </w>': 926,
 'p o s t</w>': 26,
 'm e s s a g e</w>': 54,
 'T B </w>': 6,
 'a d o p t i o n </w>': 1,
 'i s</w>': 1257,
 'g o o d </w>': 158,
 'i d e a</w>': 56,
 'p a t i e n t</w>': 3,
 'a r e</w>': 356,
 'p r o n e</w

In [ ]:
# Automate BPE
import sys
sys.path.append('./')
from minbpe import BasicTokenizer

vocab_size=1234
tokenizer = BasicTokenizer()
tokenizer.train(text_seq,vocab_size)
vocab=tokenizer.vocab
vocab
tokenizer.encode('Shree ganesh')
# isko todega phir token wala no assign karega
tokenizer.decode([83, 104, 524, 103, 271, 364, 104])
max_vocab_id = list(tokenizer.vocab.keys())[-1]
print(max_vocab_id)
tokenizer.special_tokens = {
    "<|startoftext|>": max_vocab_id + 1,
    "<|separator|>": max_vocab_id + 2,
    "<|endoftext|>": max_vocab_id + 3,
    "<|unk|>": max_vocab_id + 4,
    "<|padding|>": max_vocab_id + 5,
}

# for fine tuning special tokens
# output
print(len(tokenizer.encode(text_seq)))
tokenizer.save(file_prefix="./output/my_tokenizer")

In [ ]:
def get_vocab_size(tokenizer:BasicTokenizer):
    vocab_size=len(tokenizer.vocab)
    special_token=len(tokenizer.special_tokens)
    return vocab_size+special_token